In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data_new = pd.read_excel('output2.xlsx')
data_new.head()

,comment_id,create_time,note_id,content,user_id,sub_comment_count,like_count,topic,sentiment,type
0,68eb404500000000380293b5,1760247878000,68ea0b460000000003010908,1、你持续数年的自我对话本身也是一种行动。ai错了。2、质疑是科学精神的内核，质疑本身没错。...,62c6d4b3000000000303df86,444,8460,其他,疑问,问题反馈
1,68ef004d0000000039007c56,1760493646000,68ea0b460000000003010908,评论区让我放心了，人类不会被AI取代。[笑哭R]当人们出现迷茫或者对事物有不清楚的认知时，会...,6196655a0000000010008eac,12,590,其他,赞美,纯粹表扬
2,68ef702700000000370128a9,1760522281000,68ea0b460000000003010908,别太把ai的评价当回事儿，ai说我特别有哲学思维，很有洞见，总之就是很牛逼。给你看一下我导师...,5927beea50c4b47e9573156a,16,478,API/集成,中性,无关
3,68f014080000000039024c38,1760564232000,68ea0b460000000003010908,很同意此刻的最高赞，列出了六条有逻辑、循序渐进的comment。很高兴看到作者也很赞成。但是...,6107f95e000000000100b2d6,3,151,其他,中性,无关
4,68efd60700000000380125bb,1760548360000,68ea0b460000000003010908,人们什么时候才能明白，ai只是你给出的信息的镜子，而不是你这个人完整的本身。,5c62f55f000000001202c238,2,146,其他,中性,无关


In [4]:
data_new.shape

(746, 10)

In [4]:
data_filter1 = data_new[data_new['topic'].isin(['竞品对比'])]
data_filter1.head()

,comment_id,create_time,note_id,content,user_id,sub_comment_count,like_count,topic,sentiment,type
21,66a61164000000000401895a,1722159460000,662b00480000000001033236,博主这个方法很像康奈尔笔记。幕布是记录，flomo是思考notion收总结,65d403b9000000000d027d67,7,115,竞品对比,中性,无关
25,66e0cd1e0000000004030762,1726008607000,662b00480000000001033236,感谢分享，看完的我正试图用原生的无边记（代替幕布）备忘录（代替flomo），因为感觉功能相似...,554e45daa46e9637685d8880,2,7,竞品对比,赞美,纯粹表扬
30,65e8619f000000000200e7b2,1709728159000,640860f1000000001300f4cc,宝们！作为拖延又追求完美的P人 终于发布了新的一期notion分享啦 欢迎大家去看[萌萌哒...,57b5a88682ec397a8c45112b,1,17,竞品对比,疑问,问题反馈
37,67668651000000001001b9b5,1734772306000,640860f1000000001300f4cc,你好请问可以分享下notion中文版的软件吗,5b56ab9311be1053ff9bb727,2,0,竞品对比,疑问,问题反馈
39,65a8bef80000000002026a65,1705557752000,640860f1000000001300f4cc,你好，可以分享notion的每周打卡模板吗，我也想弄一个打勾就有进度条显示的记录，谢谢,5c32046c000000000500aea8,1,2,竞品对比,痛点/抱怨,问题反馈


In [5]:
data_filter1.shape

(52, 10)

In [6]:
#竞品词典
COMPETITOR_MAP = {
    'Notion': ['notion','Notion'], 
    'Obsidian': ['obsidian', 'ob', 'obsidion'], 
    '幕布 (Mubu)': ['幕布', 'mubu'],
    'FlowUs': ['flowus'],
    '思源笔记': ['思源', 'siyuan'],
    'Evernote (印象笔记)': ['evernote', '印象笔记', '印象'],
    'Apple Notes': ['apple notes', '苹果备忘录', 'ios备忘录', '备忘录'], 
    'Roam Research': ['roam research', 'roam', 'rr'],
    'Logseq': ['logseq'],
    'Craft': ['craft'],
    'Bear (熊掌记)': ['bear', '熊掌记'],
    'OneNote': ['onenote'],
    'Nool': ['nool'],
    'inbox': ['inbox'],
    '榴莲日记': ['榴莲日记'],
    '滴答清单': ['滴答', 'ticktick'],
    '语雀':['语雀'],
    '闪念贝壳':['闪念贝壳']
    # 你可以继续往这里添加...
}

In [7]:
def extract_competitors(content):
    """
    检查一条评论，返回所有被提到的竞品的“标准名称”列表
    """
    if pd.isna(content):
        return [] # 返回一个空列表
    
    content_lower = str(content).lower()
    found_competitors = set() # 使用 set 来自动去重

    # 遍历我们的词典
    for standard_name, aliases in COMPETITOR_MAP.items():
        for alias in aliases:
            # 如果在评论中找到了别名
            if alias in content_lower:
                found_competitors.add(standard_name)
                break # 找到了就不用再检查这个竞品的其他别名了, 换下一个竞品
                
    return list(found_competitors) # 将 set 转换回 list

In [ ]:
try:
    df = data_filter1

    print("开始智能提取竞品名称...")
    
    # 核心：将函数应用到 'content' 列，生成一个新列
    df['mentioned_competitors'] = df['content'].apply(extract_competitors)

    # 保存包含新列的文件
    output_filename = "flomo_classified_ALL_with_competitors.xlsx"
    df.to_excel(output_filename, index=False)
    
    print(f"处理完成，新文件已保存为: {output_filename}")
    
    print("\n--- 竞品提及分析 ---")
    
    # 筛选出提到了竞品的评论
    df_competitors = df[df['mentioned_competitors'].apply(lambda x: len(x) > 0)]
    print(f"总共 {len(df)} 条评论中，有 {len(df_competitors)} 条提到了竞品。")

    # 这让 value_counts() 可以正确统计
    competitor_counts = df_competitors.explode('mentioned_competitors')['mentioned_competitors'].value_counts()
    
    print("\n竞品被提及次数排名:")
    print(competitor_counts)

except FileNotFoundError:
    print("错误：没有找到 'flomo_classified_ALL.xlsx'。请先运行上一步的合并脚本。")
except Exception as e:
    print(f"发生错误: {e}")

开始智能提取竞品名称...
处理完成，新文件已保存为: flomo_classified_ALL_with_competitors.xlsx

--- 竞品提及分析 ---
总共 52 条评论中，有 49 条提到了竞品。

竞品被提及次数排名:
mentioned_competitors
Notion         27
幕布 (Mubu)      17
Obsidian       12
思源笔记            6
Apple Notes     2
闪念贝壳            2
OneNote         1
滴答清单            1
Logseq          1
Nool            1
Name: count, dtype: int64


C:\Users\Bingjie Jiang\AppData\Local\Temp\ipykernel_47572\3874953919.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mentioned_competitors'] = df['content'].apply(extract_competitors)
